<a href="https://colab.research.google.com/github/SiYeon-Ju/Colab_work/blob/main/ecCGL_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/tibbdc/ecCGL1

Cloning into 'ecCGL1'...
remote: Enumerating objects: 1927, done.
remote: Counting objects: 100% (1927/1927), done.
remote: Compressing objects: 100% (1049/1049), done.
remote: Total 1927 (delta 182), reused 1888 (delta 147), pack-reused 0
Receiving objects: 100% (1927/1927), 7.84 MiB | 20.06 MiB/s, done.
Resolving deltas: 100% (182/182), done.


In [ ]:
!pip install cobra

In [10]:
import os
import sys
currentPath = os.getcwd() + '/ecCGL1'
sys.path.append(currentPath + '/script/')
from ECMpy_function import *
import copy
import math
import cobra

In [18]:
# json_model_file=f"{currentPath}/model/iCW773_irr_enz_constraint_adj_PDH.json"
json_model_file = '/content/ecCGL1/model/iCW773_irr_enz_constraint_adj_PDH.json'
enz_model=get_enzyme_constraint_model(json_model_file)
# reaction_kcat_MW_file="./analysis/reaction_kcat_MW.csv"
objlist = []
glclist = []
exlist = list(np.linspace(0.1, 0.46, 2))
exlistn = []
for i in exlist:
    i = format(i,'.2f')
    exlistn.append(float(i))
# print(exlistn)
concentration = 5.5
obj='EX_lys_L_e' # EX_lys_L_e
enzcost = pd.DataFrame()

for i in exlistn:
    cond = i

    enz_model.reactions.get_by_id('EX_glc_e').bounds=(-concentration,0)#EX_glc_e EX_fru_e EX_sucr_e EX_inost_e EX_lac_D_e EX_ac_e
    enz_model.reactions.get_by_id('EX_glc_e_reverse').bounds=(0,0)
    # enz_model.reactions.get_by_id('ATPM').bounds=(6.86,6.86)
    enz_model.reactions.get_by_id('CG_biomass_cgl_ATCC13032').bounds=(cond,cond)
    enz_model.objective=obj
    enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)
    glclist.append(str(i))
    objlist.append(enz_model_pfba_solution.fluxes[obj])

    print(-enz_model_pfba_solution.fluxes['EX_glc_e'])
    print(enz_model_pfba_solution.fluxes['EX_ac_e'])
    print(enz_model_pfba_solution.fluxes['CG_biomass_cgl_ATCC13032'])
    # fluxes_outfile = f'{currentPath}/analysis/ECMpy_solution_%s_%.2g_pfba_%.2g.csv'%(obj,cond,concentration)
    enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,enz_model_pfba_solution,fluxes_outfile,json_model_file)
    enzcost['id']= enz_model_pfba_solution.index
    enzcost.set_index('id',inplace=True)
    enzcost['E_μ%s'%(cond)] = enz_model_pfba_solution['E']

    # print(enzcost['E_%s'%(cond)] )
    print(np.sum(enz_model_pfba_solution['E']))
    print("        ")
ectest = pd.DataFrame({'ec773':glclist,'obj':objlist})
print(ectest)


folddf = pd.DataFrame()
for row in enzcost.itertuples():
    if re.search(row.Index,'DAPDC_num2|DAPDH|DHDPS|ASPK|PYRC|TKT2|RPE|TKT1|TALA|GND|G6PDH2r|PGL|ENO|PGK|PDH|ICDHyr|ACONTa|ACONTb|CS_num1|FUM'):
        # print(enzcost.loc(row.Index,'E_μ0.1'))
        folddf.loc[row.Index,'E_μ=0.1'] = enzcost.loc[row.Index,'E_μ0.1']
        folddf.loc[row.Index,'E_μ=0.46'] = enzcost.loc[row.Index,'E_μ0.46']
folddf['log2_foldchange'] = folddf['E_μ=0.1']/folddf['E_μ=0.46']
foldlist = []
for eachf in folddf['log2_foldchange']:
    eachf = math.log2(eachf)
    foldlist.append(eachf)
folddf['log2_foldchange'] = foldlist
folddf = folddf.sort_values(by='log2_foldchange',ascending=False)
# folddf.to_csv('./analysis/enzcost_foldchange.csv')
folddf

Infeasible: ignored